In [118]:
import re
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2 import sql
import time

DATABASE_URL = 'postgres://u7r3q8fi3oil8d:p7c4ab7923edcd720fad4e7373c00c69924b36cd5449778bcf0e5a70268616cdd@cbec45869p4jbu.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d2jp5l4pcu7iuv'
conn    = psycopg2.connect(DATABASE_URL, sslmode="require")
cursor  = conn.cursor()


In [119]:
TELE_GROUPS = {
    # "index_n_bonkbot_present": -1002022757177, # -100: search exactly GetHistoryRequest, while positive number is to get cached of a user
    "index_n_gmgn_present": -1002202241417
}

print(f"{list(TELE_GROUPS.keys())}")

def clean_special_characters(value):
    # Remove or escape special characters
    if isinstance(value, str):
        # Example: Removing single quotes and other special characters
        value = re.sub(r"[\'\"\\]", "", value)
    return str(value)

['index_n_gmgn_present']


In [120]:
# token_info = [{'network':'Solana','ca':'ca123pump', 'ticker':'ticket01'},{'network':'Solana','ca':'ca456pump','ticker':'ticker02'}]

token_info = [{'network': 'test', 'ca': 'cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'ticker': 'trump', 'token_name': 'trump2024', 'description': 'trump2024', 'market_cap': 93919.63, 'liquidity_pool_value': 179235.06, 'total_holder': 48, 'gmgn_url': 'https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', 'index_n_smartwallet_bought': 0, 'index_n_gmgn_present': 0, 'index_n_bonkbot_present': 1, 'index_n_follower_twitter': None, 'index_n_subscriber_telegram': 0, 'index_n_twitter_search_result': None, 'twitter_url': None, 'telegram_url': None, 'website_url': None}]

# ---
def update_token_pool_summary_tbl(conn, cursor, token_info):
    """The function is to update the token_pool_summary table
    Args:
        token_info (list): _description_
    """
    try:
        # Prepare the insert statement dynamically
        start_time = time.time()
        queries = ""
        cols = [f"p_{item}" for item in list(token_info[0].keys())]

        # Apply SCD Type 2 for every single token, and insert into token_pool_summary table
        for row in token_info:
            # Clean data input
            values = [clean_special_characters(val) for val in row.values()]
            values = list(map(":=".join, zip(cols, values)))
            values = (str(values).replace(":=", ":='").replace(", '", ", ").replace("['", '').replace(']', '')).replace('"', '')

            # Prepare insert statement
            scd2_statement = ("""
                SELECT update_scd_to_token_pool_info_summary({parameters});
            """).format(
                parameters = values
            )
            queries += scd2_statement + "\n"

        # Execute
        try:
            cursor.execute(queries)
        except (psycopg2.Error) as e:
            print(f"Function `update_token_pool_summary_tbl` | Error inserting row {row}: {e}")
            conn.rollwback() # Rollback the current transaction to skip the faulty row
        else:
            # Commit transaction
            conn.commit()

        print(f"{len(token_info)} records have been added to `token_pool_info_summary`.")

        # Elapse time & return
        elapsed_time = round(time.time() - start_time, 2)
        print(f"*** 5. Update `token_pool_summary` table | Elapsed time: {elapsed_time}sec")

    except (Exception) as error:
        print(f"Update `token_pool_summary_tbl` error occured with message {error}") 



update_token_pool_summary_tbl(conn, cursor, token_info)

1 records have been added to `token_pool_info_summary`.
*** 5. Update `token_pool_summary` table | Elapsed time: 0.87sec


In [114]:
token_info = [{'network': 'Solana', 'ca': 'cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'ticker': 'trump', 'token_name': 'trump2024', 'description': 'trump2024', 'market_cap': 93919.63, 'liquidity_pool_value': 179235.06, 'total_holder': 48, 'gmgn_url': 'https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', 'index_n_smartwallet_bought': 0, 'index_n_gmgn_present': 0, 'index_n_bonkbot_present': 1, 'index_n_follower_twitter': None, 'index_n_subscriber_telegram': 0, 'index_n_twitter_search_result': None, 'twitter_url': None, 'telegram_url': None, 'website_url': None}]

# Need to convert it to : [(p_network:='Solana', p_ca:='ca123pump'), (p_network:='Solana', p_ca:='ca456pump')]
def clean_special_characters(value):
    # Remove or escape special characters
    if isinstance(value, str):
        # Example: Removing single quotes and other special characters
        value = re.sub(r"[\'\"\\]", "", value)
    return str(value)

cols = [f"p_{item}" for item in list(token_info[0].keys())]
values = []

for row in token_info:
    values = [clean_special_characters(val) for val in row.values()]
    print(f"1: {values}")
    values = list(map(":=".join, zip(cols, values)))
    print(f"2: {values}")
    values = (str(values).replace(":=", ":='").replace(", '", ", ").replace("['", '').replace(']', '')).replace('"', '')
    print(f"3: {values}")
    

1: ['Solana', 'cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'trump', 'trump2024', 'trump2024', '93919.63', '179235.06', '48', 'https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', '0', '0', '1', 'None', '0', 'None', 'None', 'None', 'None']
2: ['p_network:=Solana', 'p_ca:=cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'p_ticker:=trump', 'p_token_name:=trump2024', 'p_description:=trump2024', 'p_market_cap:=93919.63', 'p_liquidity_pool_value:=179235.06', 'p_total_holder:=48', 'p_gmgn_url:=https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', 'p_index_n_smartwallet_bought:=0', 'p_index_n_gmgn_present:=0', 'p_index_n_bonkbot_present:=1', 'p_index_n_follower_twitter:=None', 'p_index_n_subscriber_telegram:=0', 'p_index_n_twitter_search_result:=None', 'p_twitter_url:=None', 'p_telegram_url:=None', 'p_website_url:=None']
3: p_network:='Solana', p_ca:='cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', p_ticker:='trump', p_token_name:='trump2024', p_descripti

In [121]:
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os

DATABASE_URL = 'postgres://u7r3q8fi3oil8d:p7c4ab7923edcd720fad4e7373c00c69924b36cd5449778bcf0e5a70268616cdd@cbec45869p4jbu.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d2jp5l4pcu7iuv'
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgres

In [ ]:

class File(Base):
    __tablename__ = 'files'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    content = Column(LargeBinary)

Base.metadata.create_all(engine)

def upload_file(file_name):
    with open(file_name, 'rb') as f:
        content = f.read()
    new_file = File(name=file_name, content=content)
    session.add(new_file)
    session.commit()

def download_file(file_name, output_name=None):
    if output_name is None:
        output_name = file_name
    file = session.query(File).filter_by(name=file_name).first()
    with open(output_name, 'wb') as f:
        f.write(file.content)


In [4]:
import re
from bs4 import BeautifulSoup
import requests, json


def crawl_x_tele_from_gmgn(gmgn_url) -> dict:
    """The function to crawl tele_url, website, x_url and token info
    from GMGN site
    Args:
        gmgn_url (_type_): gmgn url
    Returns:
        dict: token information including social links
    """
    try:
        # Variables
        result = {}

        # Fetch the page content
        response = requests.get(gmgn_url)
        response.raise_for_status()  # Ensure we notice bad responses

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the script tag that contains JSON data
        script_tag = soup.find('script', type='application/json')

        # Check if script tag is found
        if script_tag:
            # Extract the JSON data from the script tag
            json_data = script_tag.string
            # Parse JSON data
            data = json.loads(json_data)
            
            # Extract token info
            token_info = data.get('props', {}).get('pageProps', {}).get('tokenInfo', {})
            if token_info:
                if 'liquidity' in token_info and (token_info['liquidity'] != 0 and token_info['liquidity'] is not None):
                    result['liquidity_pool_value'] = round(token_info['liquidity'], 2)
                if 'market_cap' in token_info and (token_info['market_cap'] != 0 and token_info['market_cap'] is not None):
                    result['market_cap'] = round(token_info['market_cap'], 2)
                if token_info['holder_count']:
                    result['total_holder'] = token_info['holder_count']

            # Extract the social_links object
            social_links = data.get('props', {}).get('pageProps', {}).get('tokenInfo', {}).get('social_links', {})
            if social_links:
                result['twitter_url'] = f"https://x.com/{social_links['twitter_username']}" if 'twitter_username' in social_links.keys() else None
                result['website_url'] = social_links["website"] if 'website' in social_links.keys() else None
                result['telegram_url']= social_links["telegram"] if 'telegram' in social_links.keys() else None

            return result

        else:
            print("Crawl x and tele func  - Script tag containing JSON data not found.")
            return None

    except (Exception) as e:
        print(f"Crawl x and tele func returned an error message: {e}")
        return None
    

token_ca = '3MUBH58LGWv8vGpTX1G529dYQm9jWcQeGi5Xf2novCJx'
gmgn_url = f'https://gmgn.ai/sol/token/{token_ca}'


r = crawl_x_tele_from_gmgn(gmgn_url)

print(r)

{'liquidity_pool_value': 263330.98, 'market_cap': 136599.87, 'total_holder': 51}
